In [101]:
# khai báo thư viện
import numpy as np
import random

In [102]:
#parameters setting
u=25
lamda=5
n_elite=4
n_closest=5


In [103]:
def create_individual(N): #tạo cá thể
    permutation = list(range(1, N+1))  # Tạo một danh sách từ 1 đến n
    random.shuffle(permutation)  # Xáo trộn hoán vị
    return np.array(permutation)

def create_population(u,N): # tạo quần thể gồm 4u cá thể
    x=[0 for x in range(4*u)]
    for i in range(4*u):
        x[i]=local_search(create_individual(N))
    a=np.array([x[i] for i in range(4*u)])
    return a

In [104]:
def local_search(individual):
    return individual

In [105]:
def fitness(i,j,population):   # tính fitness của 2 cá thể
    cost_rank=[cost(x) for x in population]  # tính rank theo độ hiệu quả
    cost_rank_i=int(np.where(np.argsort(cost_rank)==i)[0])
    cost_rank_j=int(np.where(np.argsort(cost_rank)==j)[0])
    diver_rank=[diver(x,population) for x in range(len(population))]   #tính rank theo độ đa dạng
    div_rank_i=int(np.where(np.argsort(diver_rank)==i)[0])
    div_rank_j=int(np.where(np.argsort(diver_rank)==j)[0])
    return cost_rank_i+(1-n_elite/len(population))*div_rank_i,cost_rank_j+(1-n_elite/len(population))*div_rank_j  

In [106]:
def distance(i,j): #tính khoảng cách 2 điểm
    return ((xcoord[i]-xcoord[j])**2+(ycoord[i]-ycoord[j])**2)**(1/2)

In [107]:
def timecheck(i,checkpoint,checktime,time):   # kiểm tra thời gian 
    count=0
    for j in range(max(int(checktime+distance(checkpoint,i)),int(rtime[i])),int(dtime[i]+1+service[i])):
        if time[j]:
            count+=1
        else:
            count=0
        if count==service[i]:
            return j-service[i]
    return -1

def timecheck2(i,time):
    count=0
    for j in range(int(rtime[i]),int(dtime[i])):
        for x in range(j,j+int(service[i])):
            if time[x]:
                count+=1
            else:
                count=0
            if count==service[i]:
                return j
    return -1
        

def capcheck(i,cap):   # kiểm tra tải trọng
    if demand[i]+cap<=max_cap:
        return True
    else:
        return False
    

def split1(individual):  #  decode 1 cá thể thành các route
    x=individual
    mark=[0]
    j=0
    i=0
    checkpoint=0
    checktime=int(rtime[0])
    while j<veh_num:
        cur_time = [True for _ in range(0,int(dtime[0])+1)]
        for i in range(0,int(rtime[0])):
            cur_time[i]=False
        cur_cap=0
        t=timecheck(x[i],checkpoint,checktime,cur_time)
        if t!=-1 and capcheck(x[i],cur_cap):
            cur_cap=cur_cap+demand[x[i]]
            for k in range(int(checktime),int(t+service[x[i]])):
                cur_time[k]=False
            checktime=t+service[x[i]]
            checkpoint=x[i]
            i+=1
        else:
            mark.append(i)
            j+=1
            checkpoint=0
            checktime=int(rtime[0])
        if i==len(x):
            mark.append(len(x))
            return(np.insert(x,mark,[0 for _ in range(len(mark))]))
    return np.array([-1])        

def split2(individual):
    time=[[True for _ in range(0,int(dtime[0])+1)] for _ in range(veh_num)]
    route=[[0,0] for _ in range(veh_num)]
    timecheck=[0 for _ in range(veh_num)]
    cur_cap=[0 for _ in range(veh_num)]
    for i in range(cus_num):
        for j in range(veh_num):
            a=False
            for k in range(len(route[j])):
                t=timecheck2(int(individual[i]),time[j]) 
                if t!=-1 and capcheck(individual[i],int(cur_cap[j])):
                    cur_cap[j]=cur_cap[j]+demand[individual[i]]
                    for k1 in range(int(t),int(t+service[x[i]])):
                        time[j][k1]=False
                    route[j].insert(k,individual[i])
                    a=True
                    break
            if a==True:
                break
        if a==False:
            return np.array([-1])
    result=[0]
    for i in range(veh_num):
        result.extend(route[i][1:])
    return result
        

In [108]:
def cost(individual):  # tính tổng đường đi của 1 cá thể
    x=split2(individual)
    if x[0]!=-1:
        sum=0
        for i in range(len(x)-1):
            sum+=distance(x[i],x[i+1])
        return sum
    else:
        return float('inf')

In [109]:
def broken_pairs_distance(individual1,individual2): # tính khoảng cách broken-pairs 
   dist=0
   x1=np.insert(individual1,[0,len(individual1)],[0,0])
   x2=np.insert(individual2,[0,len(individual2)],[0,0])
   for i in range(1,cus_num+1):
      t1=int(np.where(individual1==i)[0])
      t2=int(np.where(individual2==i)[0])
      if (x1[t1-1]==x2[t2-1]) or (x1[t1+1]==x2[t2-1]) or (x1[t1-1]==x2[t2+1]) or (x1[t1+1]==x2[t2+1]):
         dist+=1
   return dist

In [110]:
def diver(i,population):   # tính độ đa dạng dựa trên broken-pairs
    x=[]
    for j in range(len(population)):
        if j!=i:
            x.append(broken_pairs_distance(population[i],population[j]))
    return sum(sorted(x, reverse=True)[:n_closest])

In [111]:
def selection(population):   # chọn 2 cá thể bằng tournament và crossover
    a=[0,0]
    for i in range(2):
        x=random.randint(0, len(population)-1)
        y=random.randint(0, len(population)-1)
        x1,y1=fitness(x,y,population)
        if x1<=y1:
            a[i]=x
        else:
            a[i]=y
    return ordered_crossover(population[a[0]],population[a[1]])

In [112]:
def ordered_crossover(parent1, parent2):   # OX crosssover
    parent1=list(parent1)
    parent2=list(parent2)
    n=len(parent1)
    child=[0 for x in range(len(parent1))]

    # Chọn một đoạn ngẫu nhiên từ parent1
    start_index = random.randint(0, len(parent1) - 1)
    end_index = random.randint(start_index + 1, len(parent1))
    
    # Tạo một con mới với gen từ đoạn đã chọn
    child[start_index:end_index] = parent1[start_index:end_index]

    # Hoàn thành con với gen từ parent2 theo thứ tự xuất hiện
    index1=end_index
    index2=end_index
    for _ in range(n):
        if index1==n:
            index1=0    
        if parent2[index1] not in child:
            if index2==n:
                index2=0
            child[index2]=parent2[index1]
            index2+=1
        index1+=1
    return np.array(child)

In [113]:
#Lấy dữ liệu
xcoord=np.array([])
ycoord=np.array([])
demand=np.array([])
rtime=np.array([])
dtime=np.array([])
service=np.array([])

data = open('C101.txt','r')
lines = data.readlines()
for i in range(len(lines)):
    if lines[i]=='NUMBER     CAPACITY\n':
        veh_num,max_cap=map(int,lines[i+1].strip().split())
    if lines[i]=='CUSTOMER\n':
        j=i+3
        while j<len(lines):
            a,b,c,d,e,f,g=map(int,lines[j].strip().split())
            xcoord=np.append(xcoord,b)
            ycoord=np.append(ycoord,c)
            demand=np.append(demand,d)
            rtime=np.append(rtime,e)
            dtime=np.append(dtime,f)
            service=np.append(service,g)
            j+=1
cus_num=len(demand)-1
# Khởi tạo quần thể
population=create_population(u,cus_num)
feasible_population=[]
infeasible_population=[]
for x in population:
    print('t')
    if cost(x)==float('inf'):
        infeasible_population.append(list(x))
    else:
        feasible_population.append(list(x))
infeasible_population=np.array(infeasible_population)
feasible_population=np.array(feasible_population)

# vòng lặp
for t in range(lamda):
    print('epoch:',t)
    new_indiv=selection(population)
    new_indiv=local_search(new_indiv)
    if cost(new_indiv)==float('inf'):
        random_number = random.randint(0, 1)
        if random_number==0:
            population=np.vstack((population,new_indiv))
            infeasible_population=np.vstack((infeasible_population,new_indiv))
        else:
            new_indiv=local_search(new_indiv)
            if cost(new_indiv)==float('inf'):
                population=np.vstack((population,new_indiv))
                infeasible_population=np.vstack((infeasible_population,new_indiv))
            else:
                population=np.vstack((population,new_indiv))
                feasible_population=np.vstack((feasible_population,new_indiv))
    else:
        population=np.vstack((population,new_indiv))
        feasible_population=np.vstack((feasible_population,new_indiv))
result=float('inf')
for x in feasible_population:
    if cost(x)<result:
        result=cost(x)
print(result)



t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
t
epoch: 0
epoch: 1
epoch: 2
epoch: 3
epoch: 4
3280.0236465267735
